## Обработка фида, тестирование на корректность значений
### Задача
1. Числовые значения в фиде заменить на текстовые значения из файла РМС, в следующих колонках:
    * __ga:dimension5__ (Отдел)
    * __ga:dimension6__ (Подотдел)
2. Проверка фида по условиям:
    * Проверку уникальности строк
    * Проверку количества "столбцов"

### Результаты  
* __Данные на входе:__ 2 файла - feed и rms, указываем путь до папки
* __Данные на выходе:__ обработанный фид в формате csv, название файла: feed_updated
* __Результаты проверки скрипта на корректность__ печатаются в командной строке/ipynb

In [171]:
import pandas as pd
import numpy as np
import os
import xlsxwriter
import glob
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)
import warnings
warnings.filterwarnings('ignore')

In [192]:
#указываем путь до папки с файлами
path = '/Users/peter/Desktop/feed_new/'

In [195]:
#параметры для проверки фида
parametres_feed = {'columns_number':23,
                   
                   'columns_names':       
['ga:productSku','ga:productCategoryHierarchy','ga:dimension131','ga:dimension132','ga:productName','ga:dimension5',
 'ga:dimension6','ga:productBrand','ga:dimension9','ga:dimension11','ga:dimension55','ga:dimension57',
 'ga:dimension59','ga:dimension60','ga:dimension61','ga:dimension62','ga:dimension63','ga:dimension64',
 'ga:dimension65','ga:dimension77','ga:dimension110','ga:dimension111','ga:dimension113'],
                  
                   'not_string_columns':
['ga:productSku','ga:productCategoryHierarchy','ga:dimension131',
    'ga:dimension57','ga:dimension61','ga:dimension62','ga:dimension63','ga:dimension110','ga:dimension111']}

In [193]:
def get_files(path=path):
    '''
    Фид в csv файле, выгрузка РМС в exlsx файле
    '''
    allert_incorrect_count = 'Некорректное количество файлов в папке'
    dict_csv = {}
    dict_xlsx = {}
    for filename in glob.glob(os.path.join(path,'*.csv')):
        name = (filename.split('/')[-1])
        dict_csv[name] = filename
    if len(dict_csv)>1:
        print(f'{allert_incorrect_count}, Файлов сsv:{len(dict_csv)}')
    else:
        print('ФИД загружен')
    for filename in glob.glob(os.path.join(path,'*.xlsx')):
        name = filename.split('/')[-1]
        dict_xlsx[name] = filename
    if len(dict_xlsx)>1:
        print(f'{allert_incorrect_count}, Файлов xlsx:{len(dict_xlsx)}')
    else:
        print('РМС загружен')
    return dict_csv, dict_xlsx

In [194]:
feed_dict, rms_dict = get_files()

ФИД загружен
РМС загружен


In [178]:
def get_otdel_podotdel(rms_dict = rms_dict):
    df_rms = pd.read_excel(list(rms_dict.values())[0])
    df_podotdel = ((df_rms[[i for i in df_rms if i.lower().replace('%','') in ['подотдел']]]\
                   .drop_duplicates())\
                   .iloc[:-1,:])
    df_otdel = df_rms[[i for i in df_rms if i.lower().replace('%','') in ['отдел']]].drop_duplicates().iloc[:-1,:]
    df_otdel['%Отдел'] = df_otdel['%Отдел'].astype('int16')
    df_podotdel['%Подотдел'] = df_podotdel['%Подотдел'].astype('int16')
    display()
    return df_otdel, df_podotdel

In [92]:
#a,b = get_otdel_podotdel() - удалить

In [182]:
def merge_otdel_podotdel(feed_dict = feed_dict):
    df_feed = pd.read_csv(list(feed_dict.values())[0])
    df_otdel, df_podotdel = get_otdel_podotdel()
    df = df_feed.merge(df_podotdel, right_on='%Подотдел', left_on='ga:dimension6', how='left')
    df = df.merge(df_otdel, right_on='%Отдел', left_on='ga:dimension5', how='left')
    df['ga:dimension5'] = df['Отдел']
    df['ga:dimension6'] = df['Подотдел']
    df.drop([i for i in df.columns if ('отдел' or 'подотдел') in i.lower()], axis=1, inplace=True)
    df.to_csv(path+'feed_new.csv', index = False)
    print('Фид сохранен в папку')
    display(df.head(5))
    return df

In [183]:
df = merge_otdel_podotdel()

Фид сохранен в папку


,ga:productSku,ga:productCategoryHierarchy,ga:dimension131,ga:dimension132,ga:productName,ga:dimension5,ga:dimension6,ga:productBrand,ga:dimension9,ga:dimension11,ga:dimension55,ga:dimension57,ga:dimension59,ga:dimension60,ga:dimension61,ga:dimension62,ga:dimension63,ga:dimension64,ga:dimension65,ga:dimension77,ga:dimension110,ga:dimension111,ga:dimension113
0,18534166,5,20,not set,Стеллаж 25 секций 170.8x170.8x31 см. ЛДСП. цве...,ХРАНЕНИЕ,СТЕЛЛАЖИ НАПОЛЬНЫЕ,not set,B,Россия,Белый,0,not set,not set,173.0,32.0,23.0,ЛДСП,STD,not set,0,85.0,LeroyMerlin
1,18079634,5,20,not set,Стеллаж 3 секции 36x106x31 см. ЛДСП. цвет дуб ...,ХРАНЕНИЕ,СТЕЛЛАЖИ НАПОЛЬНЫЕ,NO NAME,A,Россия,Дуб сонома,0,not set,not set,105.0,33.0,9.0,ЛДСП,STD,not set,0,16.1,LeroyMerlin
2,18079685,5,20,not set,Стеллаж 8 секций 70x137.2x31.5 см. ЛДСП. цвет ...,ХРАНЕНИЕ,СТЕЛЛАЖИ НАПОЛЬНЫЕ,not set,A,Россия,Венге,0,not set,not set,140,33,12.5,ЛДСП,STD,not set,0,31.5,LeroyMerlin
3,18079651,5,20,not set,Стеллаж 8 секций 70x137x31 см. ЛДСП. цвет дуб ...,ХРАНЕНИЕ,СТЕЛЛАЖИ НАПОЛЬНЫЕ,NO NAME,A,Россия,Дуб сонома,0,not set,not set,140,33,12.5,ЛДСП,STD,not set,0,31.5,LeroyMerlin
4,18079669,5,20,not set,Стеллаж 3 секции 36.4x103.6x31.5 см. ЛДСП. цве...,ХРАНЕНИЕ,СТЕЛЛАЖИ НАПОЛЬНЫЕ,not set,A,Россия,Венге,0,not set,not set,105.0,33.0,9.0,ЛДСП,STD,not set,0,16.1,LeroyMerlin


In [196]:
def checking_correct_values_in_columns():
    global df, parametres_feed
    error_list = []
    incorrect_values_dict = {}
    for col in list(df.columns):
        if col in parametres_feed['not_string_columns']:
            for val in list(df[col].unique()): #если значения явно не стринг, float(val) - ок, нет - ошибка
                try:
                    float(val)
                except Exception as e:
                    error_list.append(val)
            if len(error_list) > 0:
                incorrect_values_dict[col] = error_list
                error_list = []
        else:
            for val in list(df[col].unique()): #если значение явно стринг, float(val) - не ок, нет - все ок
                try:
                    float(val)
                    error_list.append(val)
                except Exception as e:
                    continue
            if len(error_list) > 0:
                incorrect_values_dict[col] = error_list
                error_list = []
    if len(incorrect_values_dict.keys())>0:
        print('В столбцах есть некорректные значения, не соответствующие типу столбца')
        return incorrect_values_dict
    else:
        print('Типы значений в столбцах: OK')

In [197]:
checking_correct_values_in_columns()

В столбцах есть некорректные значения, не соответствующие типу столбца


{'ga:productBrand': ['216528'],
 'ga:dimension55': ['0',
  '2700',
  '3000',
  '2750',
  '4000',
  '8017',
  '6500',
  '4500',
  '9016\xa0',
  '9016',
  '9601',
  '9010',
  '80',
  '6005',
  '4200'],
 'ga:dimension61': ['not set'],
 'ga:dimension62': ['not set'],
 'ga:dimension63': ['not set'],
 'ga:dimension64': ['190', '280', '16', '20', '300']}

In [190]:
def checking_feed(parametres_feed=parametres_feed):
    '''
    Проверка уникальности строк и количества "столбцов"
    '''
    global df
    
    if parametres_feed['columns_number'] != df_test.shape[1]:
        print(f'Количество столбцов не совпадает с параметрами, {df_test.shape[1]}')
    else:
        print(f'Количество столбцов: ОК')
    
    if parametres_feed['columns_names'] != list(df_test.columns):
        print(f'Наименования столбцов не совпадает с параметрами\n{list(df_test.columns)}')
    else:
        print(f'Наименования столбцов: ОК')
        
    if df_test.duplicated().sum() != 0:
        print(f'В фиде есть дубликаты строк')
    else:
        print('Дубликаты строк: ОК')

In [191]:
checking_feed()

Количество столбцов: ОК
Наименования столбцов: ОК
Дубликаты строк: ОК
